In [4]:
# importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.preprocessing import MinMaxScaler
import joblib

In [5]:
# -------------------------------
# STEP 1: LOAD AND PREPROCESS DATA
# -------------------------------

# Load the dataset (update the filename accordingly)
df = pd.read_csv("irrigation_machine.csv")


In [6]:
# first 5 rows to be printed, df.tail()
df.head()

Unnamed: 0  sensor_0  sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  \
0           0       1.0       2.0       1.0       7.0       0.0       1.0   
1           1       5.0       1.0       3.0       5.0       2.0       2.0   
2           2       3.0       1.0       4.0       3.0       4.0       0.0   
3           3       2.0       2.0       4.0       3.0       5.0       0.0   
4           4       4.0       3.0       3.0       2.0       5.0       1.0   

   sensor_6  sensor_7  sensor_8  ...  sensor_13  sensor_14  sensor_15  \
0       1.0       4.0       0.0  ...        8.0        1.0        0.0   
1       1.0       2.0       3.0  ...        4.0        5.0        5.0   
2       1.0       6.0       0.0  ...        3.0        3.0        1.0   
3       3.0       2.0       2.0  ...        4.0        1.0        1.0   
4       3.0       1.0       1.0  ...        1.0        3.0        2.0   

   sensor_16  sensor_17  sensor_18  sensor_19  parcel_0  parcel_1  parcel_2  
0        2.0        1.0        9.0        2.0         0         1         0  
1        2.0        2.0        2.0        7.0         0         0         0  
2        0.0        3.0        1.0        0.0         1         1         0  
3        4.0        1.0        3.0        2.0         0         0         0  
4        2.0        1.0        1.0        0.0         1         1         0  

[5 rows x 24 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2000 non-null   int64  
 1   sensor_0    2000 non-null   float64
 2   sensor_1    2000 non-null   float64
 3   sensor_2    2000 non-null   float64
 4   sensor_3    2000 non-null   float64
 5   sensor_4    2000 non-null   float64
 6   sensor_5    2000 non-null   float64
 7   sensor_6    2000 non-null   float64
 8   sensor_7    2000 non-null   float64
 9   sensor_8    2000 non-null   float64
 10  sensor_9    2000 non-null   float64
 11  sensor_10   2000 non-null   float64
 12  sensor_11   2000 non-null   float64
 13  sensor_12   2000 non-null   float64
 14  sensor_13   2000 non-null   float64
 15  sensor_14   2000 non-null   float64
 16  sensor_15   2000 non-null   float64
 17  sensor_16   2000 non-null   float64
 18  sensor_17   2000 non-null   float64
 19  sensor_18   2000 non-null  

In [8]:
df.columns

Index(['Unnamed: 0', 'sensor_0', 'sensor_1', 'sensor_2', 'sensor_3',
       'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9',
       'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
       'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19',
       'parcel_0', 'parcel_1', 'parcel_2'],
      dtype='object')

In [9]:
df = df.drop('Unnamed: 0', axis=1)
df.head()

sensor_0  sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  \
0       1.0       2.0       1.0       7.0       0.0       1.0       1.0   
1       5.0       1.0       3.0       5.0       2.0       2.0       1.0   
2       3.0       1.0       4.0       3.0       4.0       0.0       1.0   
3       2.0       2.0       4.0       3.0       5.0       0.0       3.0   
4       4.0       3.0       3.0       2.0       5.0       1.0       3.0   

   sensor_7  sensor_8  sensor_9  ...  sensor_13  sensor_14  sensor_15  \
0       4.0       0.0       3.0  ...        8.0        1.0        0.0   
1       2.0       3.0       1.0  ...        4.0        5.0        5.0   
2       6.0       0.0       2.0  ...        3.0        3.0        1.0   
3       2.0       2.0       5.0  ...        4.0        1.0        1.0   
4       1.0       1.0       2.0  ...        1.0        3.0        2.0   

   sensor_16  sensor_17  sensor_18  sensor_19  parcel_0  parcel_1  parcel_2  
0        2.0        1.0        9.0        2.0         0         1         0  
1        2.0        2.0        2.0        7.0         0         0         0  
2        0.0        3.0        1.0        0.0         1         1         0  
3        4.0        1.0        3.0        2.0         0         0         0  
4        2.0        1.0        1.0        0.0         1         1         0  

[5 rows x 23 columns]

In [12]:
df.describe() # Statistics of the dataset

sensor_0     sensor_1     sensor_2     sensor_3     sensor_4  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000   
mean      1.437000     1.659000     2.654500     2.674500     2.887500   
std       1.321327     1.338512     1.699286     1.855875     1.816451   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     1.000000     1.000000     1.000000     2.000000   
50%       1.000000     1.000000     2.000000     2.000000     3.000000   
75%       2.000000     2.000000     4.000000     4.000000     4.000000   
max       8.000000     9.000000    10.000000    11.000000    12.000000   

          sensor_5     sensor_6     sensor_7     sensor_8     sensor_9  ...  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000  ...   
mean      1.411000     3.315500     4.201500     1.214000     1.901000  ...   
std       1.339394     2.206444     2.280241     1.386782     1.518668  ...   
min       0.000000     0.000000     0.000000     0.000000     0.000000  ...   
25%       0.000000     2.000000     3.000000     0.000000     1.000000  ...   
50%       1.000000     3.000000     4.000000     1.000000     2.000000  ...   
75%       2.000000     5.000000     6.000000     2.000000     3.000000  ...   
max       7.000000    13.000000    12.000000     8.000000     9.000000  ...   

         sensor_13    sensor_14    sensor_15    sensor_16    sensor_17  \
count  2000.000000  2000.000000  2000.000000  2000.000000  2000.000000   
mean      2.731500     3.416000     1.206500     2.325000     1.729500   
std       1.774537     1.960578     1.258034     1.715181     1.561265   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       1.000000     2.000000     0.000000     1.000000     0.000000   
50%       2.000000     3.000000     1.000000     2.000000     1.000000   
75%       4.000000     5.000000     2.000000     3.000000     3.000000   
max      11.000000    11.000000     6.000000    10.000000    11.000000   

        sensor_18    sensor_19    parcel_0     parcel_1     parcel_2  
count  2000.00000  2000.000000  2000.00000  2000.000000  2000.000000  
mean      2.27450     1.813500     0.63550     0.730500     0.212000  
std       1.67169     1.469285     0.48141     0.443811     0.408827  
min       0.00000     0.000000     0.00000     0.000000     0.000000  
25%       1.00000     1.000000     0.00000     0.000000     0.000000  
50%       2.00000     2.000000     1.00000     1.000000     0.000000  
75%       3.00000     3.000000     1.00000     1.000000     0.000000  
max      10.00000     7.000000     1.00000     1.000000     1.000000  

[8 rows x 23 columns]

In [13]:
# -------------------------------
# STEP 2: DEFINE FEATURES AND LABELS
# -------------------------------

X = df.iloc[:, 0:20]   # This gives you columns 0 to 19 (sensor_0 to sensor_19)


y = df.iloc[:, 20:]

In [14]:
X.sample(10)

sensor_0  sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  \
543        2.0       2.0       4.0       8.0       3.0       0.0       0.0   
24         1.0       3.0       3.0       3.0       6.0       1.0       2.0   
1053       2.0       5.0       1.0       2.0       1.0       1.0       2.0   
312        1.0       1.0       4.0       2.0       0.0       3.0       4.0   
133        4.0       2.0       1.0       1.0       3.0       0.0       5.0   
226        1.0       0.0       9.0       3.0       4.0       1.0       9.0   
1981       3.0       1.0       4.0       1.0       1.0       1.0       2.0   
1107       1.0       3.0       0.0       4.0       4.0       3.0       3.0   
1533       1.0       3.0       6.0       4.0       3.0       0.0       4.0   
518        3.0       4.0       4.0       2.0       3.0       2.0       3.0   

      sensor_7  sensor_8  sensor_9  sensor_10  sensor_11  sensor_12  \
543        3.0       2.0       5.0        3.0        7.0        7.0   
24         5.0       3.0       2.0        5.0        5.0        2.0   
1053       4.0       0.0       2.0        6.0        6.0        3.0   
312        2.0       2.0       1.0        6.0        0.0        3.0   
133        4.0       0.0       3.0        2.0        2.0        6.0   
226        6.0       0.0       0.0        1.0        3.0        3.0   
1981       4.0       1.0       0.0        3.0        1.0        7.0   
1107       0.0       3.0       1.0        2.0        5.0        5.0   
1533       5.0       1.0       2.0        2.0        4.0        3.0   
518        5.0       2.0       4.0        5.0        1.0        2.0   

      sensor_13  sensor_14  sensor_15  sensor_16  sensor_17  sensor_18  \
543         3.0        4.0        2.0        3.0        0.0        5.0   
24          3.0        3.0        1.0        3.0        0.0        1.0   
1053        6.0        3.0        1.0        1.0        1.0        5.0   
312         4.0        3.0        1.0        3.0        1.0        0.0   
133         0.0        3.0        0.0        2.0        4.0        2.0   
226         3.0        2.0        2.0        3.0        8.0        0.0   
1981        2.0        4.0        1.0        2.0        2.0        2.0   
1107        1.0        4.0        0.0        2.0        2.0        3.0   
1533        1.0        2.0        4.0        3.0        3.0        1.0   
518         1.0        2.0        2.0        2.0        0.0        6.0   

      sensor_19  
543         1.0  
24          0.0  
1053        1.0  
312         1.0  
133         4.0  
226         1.0  
1981        3.0  
1107        0.0  
1533        2.0  
518         2.0

In [15]:
y.sample(10)

parcel_0  parcel_1  parcel_2
1959         0         1         0
980          0         1         0
1658         0         0         0
1053         0         1         0
269          0         0         0
1512         1         0         0
1905         0         1         0
425          0         0         0
866          0         1         0
720          1         1         1

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sensor_0   2000 non-null   float64
 1   sensor_1   2000 non-null   float64
 2   sensor_2   2000 non-null   float64
 3   sensor_3   2000 non-null   float64
 4   sensor_4   2000 non-null   float64
 5   sensor_5   2000 non-null   float64
 6   sensor_6   2000 non-null   float64
 7   sensor_7   2000 non-null   float64
 8   sensor_8   2000 non-null   float64
 9   sensor_9   2000 non-null   float64
 10  sensor_10  2000 non-null   float64
 11  sensor_11  2000 non-null   float64
 12  sensor_12  2000 non-null   float64
 13  sensor_13  2000 non-null   float64
 14  sensor_14  2000 non-null   float64
 15  sensor_15  2000 non-null   float64
 16  sensor_16  2000 non-null   float64
 17  sensor_17  2000 non-null   float64
 18  sensor_18  2000 non-null   float64
 19  sensor_19  2000 non-null   float64
dtypes: float

In [17]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   parcel_0  2000 non-null   int64
 1   parcel_1  2000 non-null   int64
 2   parcel_2  2000 non-null   int64
dtypes: int64(3)
memory usage: 47.0 KB


In [19]:
X

sensor_0  sensor_1  sensor_2  sensor_3  sensor_4  sensor_5  sensor_6  \
0          1.0       2.0       1.0       7.0       0.0       1.0       1.0   
1          5.0       1.0       3.0       5.0       2.0       2.0       1.0   
2          3.0       1.0       4.0       3.0       4.0       0.0       1.0   
3          2.0       2.0       4.0       3.0       5.0       0.0       3.0   
4          4.0       3.0       3.0       2.0       5.0       1.0       3.0   
...        ...       ...       ...       ...       ...       ...       ...   
1995       4.0       1.0       2.0       2.0       1.0       1.0       1.0   
1996       1.0       3.0       3.0       3.0       2.0       2.0       3.0   
1997       1.0       3.0       3.0       1.0       1.0       4.0       8.0   
1998       2.0       1.0       0.0       2.0       2.0       0.0       1.0   
1999       0.0       1.0       4.0       1.0       2.0       2.0       6.0   

      sensor_7  sensor_8  sensor_9  sensor_10  sensor_11  sensor_12  \
0          4.0       0.0       3.0        1.0        3.0        6.0   
1          2.0       3.0       1.0        3.0        2.0        2.0   
2          6.0       0.0       2.0        3.0        2.0        4.0   
3          2.0       2.0       5.0        3.0        1.0        2.0   
4          1.0       1.0       2.0        4.0        5.0        3.0   
...        ...       ...       ...        ...        ...        ...   
1995       2.0       1.0       2.0        4.0        3.0        3.0   
1996       3.0       1.0       5.0        2.0        2.0        4.0   
1997       1.0       0.0       0.0        3.0        2.0        4.0   
1998       3.0       0.0       0.0        0.0        5.0        2.0   
1999       8.0       5.0       1.0        2.0        4.0        3.0   

      sensor_13  sensor_14  sensor_15  sensor_16  sensor_17  sensor_18  \
0           8.0        1.0        0.0        2.0        1.0        9.0   
1           4.0        5.0        5.0        2.0        2.0        2.0   
2           3.0        3.0        1.0        0.0        3.0        1.0   
3           4.0        1.0        1.0        4.0        1.0        3.0   
4           1.0        3.0        2.0        2.0        1.0        1.0   
...         ...        ...        ...        ...        ...        ...   
1995        1.0        2.0        3.0        2.0        1.0        1.0   
1996        3.0        3.0        0.0        1.0        0.0        6.0   
1997        2.0        3.0        4.0        4.0        4.0        1.0   
1998        2.0        4.0        0.0        2.0        0.0        3.0   
1999        2.0        1.0        1.0        0.0        5.0        2.0   

      sensor_19  
0           2.0  
1           7.0  
2           0.0  
3           2.0  
4           0.0  
...         ...  
1995        0.0  
1996        2.0  
1997        0.0  
1998        0.0  
1999        1.0  

[2000 rows x 20 columns]

In [18]:
X.shape, y.shape

((2000, 20), (2000, 3))

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
# -------------------------------
# STEP 3: TRAIN-TEST SPLIT
# -------------------------------

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# -------------------------------
# STEP 4: TRAIN CLASSIFIER
# -------------------------------

# Use MultiOutputClassifier to handle multi-label classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Custom hyperparameters for RandomForest
rf = RandomForestClassifier(
    n_estimators=200,         # Number of trees
    max_depth=10,             # Maximum depth of each tree
    min_samples_split=4,      # Minimum samples to split a node
    min_samples_leaf=2,       # Minimum samples per leaf
    max_features='sqrt',      # Number of features to consider at each split ('auto', 'sqrt', 'log2', or int)
    random_state=42
)

# Wrap it with MultiOutputClassifier
model = MultiOutputClassifier(rf)

# Train the model
model.fit(X_train, y_train)


In [ ]:
# -------------------------------
# STEP 5: EVALUATE MODEL
# -------------------------------

y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y.columns))

In [ ]:
print(df[['parcel_0', 'parcel_1', 'parcel_2']].sum())

In [ ]:
import matplotlib.pyplot as plt

# Define parcel activation conditions with descriptive labels
conditions = {
    "Parcel 0 ON": df['parcel_0'],
    "Parcel 1 ON": df['parcel_1'],
    "Parcel 2 ON": df['parcel_2'],
    "Parcel 0 & 1 ON": df['parcel_0'] & df['parcel_1'],
    "Parcel 0 & 2 ON": df['parcel_0'] & df['parcel_2'],
    "Parcel 1 & 2 ON": df['parcel_1'] & df['parcel_2'],
    "All Parcels ON": df['parcel_0'] & df['parcel_1'] & df['parcel_2'],
}

# Create vertically stacked subplots (one for each condition)
fig, axs = plt.subplots(nrows=len(conditions), figsize=(10,15), sharex=True)

# Loop through each condition to plot corresponding square wave
for ax, (title, condition) in zip(axs, conditions.items()):
    ax.step(df.index, condition.astype(int), where='post', linewidth=1, color='teal')
    ax.set_title(f"Sprinkler - {title}")
    ax.set_ylabel("Status")
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['OFF', 'ON'])
   

# Label x-axis on the last subplot
axs[-1].set_xlabel("Time Index (Row Number)")

# Plot
plt.show()


In [ ]:
# Calculate combined activity of all pumps (overlap)
any_pump_on = (df['parcel_0'] == 1) | (df['parcel_1'] == 1) | (df['parcel_2'] == 1)

plt.figure(figsize=(15, 5))

# Plot individual pump statuses
plt.step(df.index, df['parcel_0'], where='post', linewidth=2, label='Parcel 0 Pump', color='blue')
plt.step(df.index, df['parcel_1'], where='post', linewidth=2, label='Parcel 1 Pump', color='orange')
plt.step(df.index, df['parcel_2'], where='post', linewidth=2, label='Parcel 2 Pump', color='green')

plt.title("Pump Activity and Combined Farm Coverage")
plt.xlabel("Time Index (Row Number)")
plt.ylabel("Status")
plt.yticks([0, 1], ['OFF', 'ON'])
plt.legend(loc='upper right')
plt.show()


In [ ]:
import joblib
from sklearn.pipeline import Pipeline

joblib.dump(model, "Farm_Irrigation_System.pkl")